In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import torch

from models.PCAH import PCAH_Net
from utils.datasetHeads import MeshHeadsDataset, ToTensor

In [3]:
test_lungs = open("test_images_lungs.txt", 'r').read().splitlines()
test_heart = open("test_images_heart.txt", 'r').read().splitlines()

In [13]:
train_heads_files = "heads_files/test_images_heads.txt"
images = open(train_heads_files, 'r').read().splitlines()

dataset = MeshHeadsDataset(images,
                           img_path="../Chest-xray-landmark-dataset/Images",
                           label_path="../Chest-xray-landmark-dataset/landmarks",
                           heart=False
                           )

device = 'cuda:0'

In [11]:
config = {}

config['device'] = device
config['Heads'] = True
config['Lungs'] = False
config['latents'] = 32
config['inputsize'] = 1024

pcaNet = PCAH_Net(config.copy()).to(device)
pcaNet.load_state_dict(torch.load('Training/PCAH/bestMSE.pt'))
pcaNet.eval()
print('Model loaded')

Model loaded


In [15]:
sample = dataset[0]
image = sample['image']
image = image.unsqueeze(0)
image.shape

torch.Size([1, 1, 233, 512, 512])

In [17]:
out = pcaNet(image.to(device))
out.shape

torch.Size([1, 20217])

In [23]:
import vedo
faces=vedo.Mesh('/media/fmatzkin/data/franco/Code/Chest-xray-landmark-dataset/Images/CQ500-CT-12_CT Thin Plain_decimated_1perc_dfm.vtk').faces()
points = out.reshape(-1,3).detach().cpu().numpy()

restored_mesh = vedo.Mesh([points, faces])
restored_mesh.write(f'test/asd.stl')
print(f'\r saved mesh', end='')

 saved mesh